<a href="https://colab.research.google.com/github/IamHeeJ/ML_Ex/blob/main/5_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#5-2 교차 검증과 그리드 서치

##검증 세트(validation set)
- 모델을 만들고 나서 테스트 세트는 딱 한번 사용되는 것이 좋음
- 따라서 추가로 검증 세트를 또 나눔

In [ ]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [ ]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [ ]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

In [ ]:
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

In [ ]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


훈련 세트와 검증 세트의 평가 점수를 이용하여 최적화
최적화 완료 후 테스트 세트로 모델을 최종 평가

In [ ]:
#for문으로 검사하여 최적화

print(dt.score(test_input, test_target))


0.8569230769230769


**검증 세트의 문제점?**

검증 세트로 인해 훈련 세트가 줄어듬

많은 데이터를 훈련에 사용할수록 좋은 모델이 만들어짐

검증 세트를 너무 조금 떼어놓으면 검증 점수가 불안정함



##교차 검증

검증 세트를 떼어 내어 평가하는 과정을 여러번 반복함

In [ ]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.0090301 , 0.00893593, 0.00759125, 0.0103631 , 0.00769567]), 'score_time': array([0.00183105, 0.00109458, 0.00104332, 0.00143719, 0.00099349]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [ ]:
import numpy as np

print(np.mean(scores['test_score']))

0.855300214703487


주의할점

cross_validate() 함수는 훈련 세트를 섞어 폴드를 나누지 않음
섞지 않는 것이 디폴트이므로 따로 설정해줘야 함
회귀 모델일 경우 KFold 분할기를 사용하고 분류 모델일 경우 Stratified를 사용

In [ ]:
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [ ]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


##하이퍼 파라미터 튜닝
- 하이퍼 파라미터 : 모델이 학습할 수 없어서 사용자가 지정해야 하는 파라미터
- 모델 ㅍ파라미터 : 모델이 학습하는 파라미터
- 튜닝 작업 : 기본값을 그대로 사용해서 모델을 훈련할 후, 검증 세트의 점수나 교차 검증을 통해 매개변수를 조금씩 바꿔가면서 최적값을 찾음
- 문제는 한 매개변수의 최적값을 찾고 다른 매개변수의 최적값을 찾으면 안됨

**그리드 서치**
하이퍼파라미터 탐색과 교차검증을 한번에 수행
즉, cross_validate() 함수를 호출할 필요x



In [ ]:
from sklearn.model_selection import GridSearchCV #교차검증을 사용하는 그리드서치

#훈련 조건 설정
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
print(gs)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})


In [ ]:
#훈련
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [ ]:
#최적화
#best_estimator 는 자동으로 최적값을 찾음
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [ ]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [ ]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [ ]:
# 가장 높은값 출력
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [ ]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
          }

In [ ]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [ ]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [ ]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


###예제 : 결정 트리 모델에서 in_impurity_deacrease, max_depth, main_samples_split 최적값 찾기

np.range는 첫 번째 매개변수 값에서 시작하여 두번째 매개변수에 도달할 때까지 세번째 매개변수를 계속 더한 배열을 생성 즉, [0.0001, 0.0002 ... 0.0009]
range도 비슷하지만 정수만 가능
즉, [5, 6, 7, 8, 9, ..., 19]와 [2, 12, 22, 32, ... , 92]
여기서 수행할 교차 검증 횟수는 9x15x10=1350
기본 5-폴드 교차 검증을 수행하므로 만들어지는 모델의 수는 6750개


In [ ]:
params = {'min_impurity_decrease':np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split' : range(2, 100, 10)
          }

dt = DecisionTreeClassifier(random_state=42)
gs = GridSearchCV(dt, params, n_jobs=-1)
gs.fit(train_input, train_target)

print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


##랜덤 서치



In [ ]:
from scipy.stats import uniform, randint

rgen = randint(0, 10)
rgen.rvs(10) #랜덤으로 10개 추출

array([2, 3, 0, 6, 4, 6, 6, 6, 4, 8])

In [ ]:
#샘플링숫자를 1000으로 늘려서 고르게 샘플링되는지 확인
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([106, 103, 102,  98,  88, 101, 100, 108,  99,  95]))

In [ ]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.99127393, 0.7266136 , 0.14643055, 0.81053761, 0.19298653,
       0.04347114, 0.14445381, 0.42925139, 0.12568485, 0.96236701])

###예제 : 결정 트리 모델에서 min_impurity_decrease, max_depth, min_samples_split, min_samples_leaf 최적값 찾기

min_samples_leaf : 리프 노드가 되기 위한 최소 샘플의 수
어떤 노드가 분할하여 만들어질 자식 노드의 샘플 수가 이 값보다 작으면 분할 안됨
샘플링 횟수는 n_iter 매개변수에 지정

In [ ]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [ ]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [ ]:
dt = gs.best_estimator_
print(dt)
print(dt.score(test_input, test_target))

DecisionTreeClassifier(max_depth=39,
                       min_impurity_decrease=0.00034102546602601173,
                       min_samples_leaf=7, min_samples_split=13,
                       random_state=42)
0.86


**주의**
교차 검증, 그리드 서치, 랜덤 서치는 모델을 훈련하는 것이 아니라 검증하는 것임
즉, 적절한 하이퍼파라미터를 찾는데 사용해야 함
찾은 하이퍼파라미터로 훈련해야함

##확인문제

In [ ]:
gs = RandomizedSearchCV(DecisionTreeClassifier(splitter='random', random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42,
                                                    splitter='random'),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f69468bf850>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7f69468bff70>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f69468bff40>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f69468bd4e0>},
                   random_state=42)

In [ ]:
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

dt = gs.best_estimator_
print(dt.score(test_input, test_target))

{'max_depth': 43, 'min_impurity_decrease': 0.00011407982271508446, 'min_samples_leaf': 19, 'min_samples_split': 18}
0.8458726956392981
0.786923076923077
